In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Semantic Engine work

In [1]:
!pip install sentence-transformers faiss-cpu --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00:00:0100:01


In [2]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

print("Libraries installed and imported.")

# --- IMPORTANT ---
# Make sure this path points to your cleaned CSV file from the previous step.
# If you saved it in /kaggle/working/, the path will be as below.
# If you uploaded it as a new dataset, it might be /kaggle/input/your-dataset-name/articles_cleaned.csv
ARTICLES_CLEAN_PATH = '/kaggle/input/my-own/articles_cleaned.csv'

# Load the data, ensuring article_id is a string
articles_df = pd.read_csv(ARTICLES_CLEAN_PATH, dtype={'article_id': str})

print(f"Loaded {len(articles_df)} articles from the cleaned dataset.")
articles_df.head()

2025-08-31 08:03:23.609811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756627403.823197      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756627403.886967      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Libraries installed and imported.
Loaded 50431 articles from the cleaned dataset.


,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name,detail_desc
0,0108775015,Strap top,Vest top,Garment Upper body,Solid,Black,Dark,Black,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,Strap top,Vest top,Garment Upper body,Solid,White,Light,White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.
2,0110065001,OP T-shirt (Idro),Bra,Underwear,Solid,Black,Dark,Black,Clean Lingerie,Lingeries/Tights,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
3,0110065002,OP T-shirt (Idro),Bra,Underwear,Solid,White,Light,White,Clean Lingerie,Lingeries/Tights,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065011,OP T-shirt (Idro),Bra,Underwear,Solid,Light Beige,Dusty Light,Beige,Clean Lingerie,Lingeries/Tights,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [3]:
# List of text columns to combine
text_feature_columns = [
    'prod_name',
    'product_type_name',
    'product_group_name',
    'graphical_appearance_name',
    'colour_group_name',
    'perceived_colour_value_name',
    'perceived_colour_master_name',
    'department_name',
    'index_name',
    'section_name',
    'garment_group_name',
    'detail_desc'
]

# --- Pre-processing: Fill any potential missing values in these columns ---
# This is crucial to prevent errors during string concatenation.
for col in text_feature_columns:
    articles_df[col].fillna('', inplace=True)

# --- Create the combined text feature with clear separators ---
# This structured format helps the model understand the context of each word.
def create_embedding_text(row):
    return (
        f"Name: {row['prod_name']}. "
        f"Type: {row['product_type_name']} in {row['product_group_name']}. "
        f"Appearance: {row['graphical_appearance_name']} with color {row['colour_group_name']}. "
        f"Category: {row['department_name']}, {row['section_name']}. "
        f"Description: {row['detail_desc']}"
    )

articles_df['text_for_embedding'] = articles_df.apply(create_embedding_text, axis=1)

print("--- Example of the new comprehensive text feature ---")
print(articles_df[['article_id', 'text_for_embedding']].head().to_string())

/tmp/ipykernel_36/2615926203.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  articles_df[col].fillna('', inplace=True)


--- Example of the new comprehensive text feature ---
   article_id                                                                                                                                                                                                                                                                                                                                                                                   text_for_embedding
0  0108775015                                                                                                                                                                                                      Name: Strap top. Type: Vest top in Garment Upper body. Appearance: Solid with color Black. Category: Jersey Basic, Womens Everyday Basics. Description: Jersey top with narrow shoulder straps.
1  0108775044                                                                                                                               

In [4]:
# Load a pre-trained sentence transformer model
# 'all-MiniLM-L6-v2' is a great balance of speed and performance.
model = SentenceTransformer('all-MiniLM-L6-v2')

print("Generating semantic embeddings for all articles. This will take a few minutes...")
# Convert the text to embeddings
semantic_embeddings = model.encode(
    articles_df['text_for_embedding'].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True
)

print(f"\nEmbeddings generated. Shape: {semantic_embeddings.shape}")

# --- Build the FAISS Index for fast searching ---
embedding_dimension = semantic_embeddings.shape[1]
# We use IndexFlatL2, which performs exact search using L2 (Euclidean) distance.
index = faiss.IndexFlatL2(embedding_dimension)

# Add the article embeddings to the index. FAISS requires float32 data type.
index.add(semantic_embeddings.astype('float32'))

print(f"FAISS index built. Total items in index: {index.ntotal}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating semantic embeddings for all articles. This will take a few minutes...


Batches:   0%|          | 0/1576 [00:00<?, ?it/s]


Embeddings generated. Shape: (50431, 384)
FAISS index built. Total items in index: 50431


In [5]:
# Define paths for the output files
FAISS_INDEX_PATH = '/kaggle/working/semantic_faiss_index.bin'
ARTICLE_IDS_MAP_PATH = '/kaggle/working/semantic_article_ids.csv'

# Save the FAISS index to disk
faiss.write_index(index, FAISS_INDEX_PATH)

# CRITICAL: Save the article_ids in the *exact same order* as the embeddings.
# This file is our map from an index position (e.g., 42) back to the real article_id.
articles_df[['article_id']].to_csv(ARTICLE_IDS_MAP_PATH, index=False)

print(f"Successfully saved semantic engine artifacts to /kaggle/working/:")
print(f"- FAISS Index: {FAISS_INDEX_PATH}")
print(f"- Article ID Mapping: {ARTICLE_IDS_MAP_PATH}")

Successfully saved semantic engine artifacts to /kaggle/working/:
- FAISS Index: /kaggle/working/semantic_faiss_index.bin
- Article ID Mapping: /kaggle/working/semantic_article_ids.csv


In [11]:
def search_products(query, k=5):
    """Encodes a query, searches the FAISS index, and returns top k results."""
    query_vector = model.encode([query], convert_to_numpy=True).astype('float32')
    distances, indices = index.search(query_vector, k)
    
    print(f"\n--- Top {k} results for query: '{query}' ---\n")
    
    for rank, idx in enumerate(indices[0]):
        # Directly get the row from articles_df using iloc
        row = articles_df.iloc[idx]
        article_id = row['article_id']
        
        print(f"Result {rank+1}:")
        print(f"  Article ID: {article_id}")
        print(f"  Product Name: {row['prod_name']}")
        # --- ADDED THIS LINE ---
        print(f"  Color: {row['colour_group_name']}")
        # ---------------------
        print(f"  Description: {row['detail_desc']}")
        print(f"  Similarity Score (L2 Distance): {distances[0][rank]:.4f}\n")

# --- Test queries ---
# Now, when you run this, the color will be included in the output.
search_products("black dress for summer")
search_products("blue wide leg jeans")
search_products("white sneakers for men")
search_products("blue striped t shirt for men")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Top 5 results for query: 'black dress for summer' ---

Result 1:
  Article ID: 0926246001
  Product Name: Summer nights dress
  Color: Black
  Description: Short lace dress with a concealed zip at the back and long, slightly wider sleeves with narrow, covered elastication at the cuffs. Fitted at the top, a seam at the waist and a wide, circular skirt. Lined.
  Similarity Score (L2 Distance): 0.6982

Result 2:
  Article ID: 0926246003
  Product Name: Summer nights dress
  Color: Beige
  Description: Short lace dress with a concealed zip at the back and long, slightly wider sleeves with narrow, covered elastication at the cuffs. Fitted at the top, a seam at the waist and a wide, circular skirt. Lined.
  Similarity Score (L2 Distance): 0.7286

Result 3:
  Article ID: 0786242001
  Product Name: HW Black swan
  Color: Black
  Description: Fancy dress costume with a lace body and a sewn-in flared skirt in tulle. Body with narrow adjustable shoulder straps, padded cups, a zip at the back

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Top 5 results for query: 'blue wide leg jeans' ---

Result 1:
  Article ID: 0833622001
  Product Name: WIDELEG LIGHT WEIGHT
  Color: Light Blue
  Description: Cropped jeans in washed, lightweight cotton denim with covered elastication at the waist and a fake fly. Wide, straight-cut legs with scalloped, hole embroidered hems.
  Similarity Score (L2 Distance): 0.5924

Result 2:
  Article ID: 0926502001
  Product Name: Hudson Wide Leg Denim
  Color: Blue
  Description: 5-pocket jeans in washed cotton denim with a high waist, zip fly and button and wide legs. The cotton content of the jeans is partly recycled.
  Similarity Score (L2 Distance): 0.5968

Result 3:
  Article ID: 0725338002
  Product Name: Empire HW wide leg denim
  Color: Yellow
  Description: 5-pocket jeans in dyed denim with a high waist, zip fly and button and straight, wide legs.
  Similarity Score (L2 Distance): 0.6087

Result 4:
  Article ID: 0781135002
  Product Name: Wide H.W Ankle
  Color: Blue
  Description: 5-p

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Top 5 results for query: 'white sneakers for men' ---

Result 1:
  Article ID: 0889712001
  Product Name: Alton Sneaker
  Color: White
  Description: Trainers in imitation leather with a padded edge and tongue, and lacing at the front. Linings and insoles in jersey made from recycled polyester, and fluted soles.
  Similarity Score (L2 Distance): 0.7355

Result 2:
  Article ID: 0728830001
  Product Name: Beta Low court Sneaker
  Color: White
  Description: Trainers with a padded top edge and lacing at the front. Mesh linings and insoles and rubber soles.
  Similarity Score (L2 Distance): 0.7416

Result 3:
  Article ID: 0766093002
  Product Name: Rhino Sneaker
  Color: White
  Description: Trainers in imitation leather with a padded top edge and tongue, and lacing at the front. Mesh linings and insoles and patterned soles.
  Similarity Score (L2 Distance): 0.7840

Result 4:
  Article ID: 0585138001
  Product Name: Red high top sneaker
  Color: Beige
  Description: Hi-tops in canvas 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Top 5 results for query: 'blue striped t shirt for men' ---

Result 1:
  Article ID: 0856395005
  Product Name: ICE striped linen tee
  Color: Dark Blue
  Description: T-shirt in striped, airy linen jersey with a round neckline.
  Similarity Score (L2 Distance): 0.7167

Result 2:
  Article ID: 0882817002
  Product Name: Slice Tee
  Color: Light Turquoise
  Description: Short-sleeved sports top in fast-drying functional fabric with a reflective print. Slightly longer and rounded at the back.
  Similarity Score (L2 Distance): 0.7236

Result 3:
  Article ID: 0882817001
  Product Name: Slice Tee
  Color: Grey
  Description: Short-sleeved sports top in fast-drying functional fabric with a reflective print. Slightly longer and rounded at the back.
  Similarity Score (L2 Distance): 0.7421

Result 4:
  Article ID: 0792899001
  Product Name: GREY 50?s pocket tee
  Color: Black
  Description: T-shirt in stretch cotton jersey with raw edges, a chest pocket and short sleeves with sewn-in turn

**Block to run while demonstration**

In [13]:
# === STEP 1: Import Libraries and Set Up Paths ===
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Define the paths to the artifacts we saved earlier
FAISS_INDEX_PATH = '/kaggle/working/semantic_faiss_index.bin'
ARTICLE_IDS_MAP_PATH = '/kaggle/working/semantic_article_ids.csv'
ARTICLES_DATA_PATH = '/kaggle/input/my-own/articles_cleaned.csv' # Needed for displaying product details

print("--- Loading necessary files ---")

# === STEP 2: Load the Artifacts into Memory ===

# Load the FAISS index
try:
    index = faiss.read_index(FAISS_INDEX_PATH)
    print(f"✓ FAISS index loaded successfully. Contains {index.ntotal} vectors.")
except Exception as e:
    print(f"✗ Error loading FAISS index: {e}")

# Load the article ID mapping file
try:
    article_id_map_df = pd.read_csv(ARTICLE_IDS_MAP_PATH, dtype={'article_id': str})
    print(f"✓ Article ID map loaded successfully. Contains {len(article_id_map_df)} IDs.")
except Exception as e:
    print(f"✗ Error loading article ID map: {e}")

# Load the full articles dataframe to get product details for display
try:
    articles_df = pd.read_csv(ARTICLES_DATA_PATH, dtype={'article_id': str})
    # Set article_id as the index for fast lookups (df.loc[...])
    articles_df.set_index('article_id', inplace=True)
    print(f"✓ Full articles data loaded successfully.")
except Exception as e:
    print(f"✗ Error loading articles data: {e}")

# Load the pre-trained Sentence Transformer model
# This must be the *exact same model* used to create the embeddings
try:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print("✓ Sentence Transformer model loaded successfully.")
except Exception as e:
    print(f"✗ Error loading Sentence Transformer model: {e}")

print("\n--- Semantic Search Engine is Ready ---")


# === STEP 3: Define the Reusable Search Function ===

def semantic_search(query: str, top_k: int = 5):
    """
    Takes a text query, embeds it, searches the FAISS index,
    and prints the details of the top_k most similar products.
    """
    if 'index' not in globals() or 'model' not in globals():
        print("Error: Search index or model not loaded.")
        return

    print(f"\n=================================================")
    print(f"| Searching for: '{query}' (Top {top_k} Results) |")
    print(f"=================================================")

    # 1. Embed the query
    query_vector = model.encode([query], convert_to_numpy=True).astype('float32')
    
    # 2. Search the FAISS index
    distances, indices = index.search(query_vector, top_k)
    
    # 3. Retrieve and display the results
    for i, idx in enumerate(indices[0]):
        # Get the article_id from our mapping dataframe
        article_id = article_id_map_df.iloc[idx]['article_id']
        
        try:
            # Get the product details from the main articles dataframe using .loc for speed
            product_details = articles_df.loc[article_id]
            
            print(f"\n--- Result #{i+1} ---")
            print(f"  Article ID: {article_id}")
            print(f"  Product Name: {product_details['prod_name']}")
            print(f"  Color: {product_details['colour_group_name']}")
            print(f"  Description: {product_details['detail_desc']}")
            print(f"  L2 Distance (Similarity): {distances[0][i]:.4f}") # Lower is better
        except KeyError:
            print(f"\n--- Result #{i+1} ---")
            print(f"  Could not find details for article_id: {article_id}")

# === STEP 4: Demonstrate with Example Queries ===
# You can add or change any of these queries to demonstrate the engine's power.

semantic_search("red floral summer dress")
semantic_search("men's dark wash slim fit jeans")
semantic_search("pink hoodie with a zipper")

--- Loading necessary files ---
✓ FAISS index loaded successfully. Contains 50431 vectors.
✓ Article ID map loaded successfully. Contains 50431 IDs.
✓ Full articles data loaded successfully.
✓ Sentence Transformer model loaded successfully.

--- Semantic Search Engine is Ready ---

| Searching for: 'red floral summer dress' (Top 5 Results) |


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Result #1 ---
  Article ID: 0733076002
  Product Name: Red Velvet
  Color: Light Pink
  Description: Sleeveless, ankle-length dress in a viscose crêpe weave with a V-neck, covered buttons down the front, a gathered seam at the waist and a straight-cut skirt that flares gently to the hem. Unlined.
  L2 Distance (Similarity): 0.6747

--- Result #2 ---
  Article ID: 0733076004
  Product Name: Red Velvet
  Color: Dark Blue
  Description: Sleeveless, ankle-length dress in a viscose crêpe weave with a V-neck, covered buttons down the front, a gathered seam at the waist and a straight-cut skirt that flares gently to the hem. Unlined.
  L2 Distance (Similarity): 0.6813

--- Result #3 ---
  Article ID: 0733076001
  Product Name: Red Velvet
  Color: Off White
  Description: Sleeveless, ankle-length dress in a viscose crêpe weave with a V-neck, covered buttons down the front, a gathered seam at the waist and a straight-cut skirt that flares gently to the hem. Unlined.
  L2 Distance (Similari

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Result #1 ---
  Article ID: 0891638003
  Product Name: Regular fit Lowprice
  Color: Dark Grey
  Description: 5-pocket jeans in washed, stretch denim with a regular waist, zip fly and button and gently tapered legs with good room for movement over the thighs and knees.
  L2 Distance (Similarity): 0.5930

--- Result #2 ---
  Article ID: 0891638002
  Product Name: Regular fit Lowprice
  Color: Dark Blue
  Description: 5-pocket jeans in washed, stretch denim with a regular waist, zip fly and button and gently tapered legs with good room for movement over the thighs and knees.
  L2 Distance (Similarity): 0.5953

--- Result #3 ---
  Article ID: 0753512001
  Product Name: Skinny Midprice No Fade Black
  Color: Black
  Description: 5-pocket jeans in stretch organic cotton denim that has been dyed multiple times for a better post-wash colour. Regular waist, zip fly and skinny legs.
  L2 Distance (Similarity): 0.6033

--- Result #4 ---
  Article ID: 0754044001
  Product Name: Athletic Skin

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Result #1 ---
  Article ID: 0907692001
  Product Name: Eclipse zip hood(1)
  Color: Black
  Description: Oversized zip-through hoodie in sweatshirt fabric with a lined, drawstring hood and zip down the front. Dropped shoulders, long sleeves and ribbing at the cuffs and hem. The polyester content of the hoodie is partly recycled.
  L2 Distance (Similarity): 0.6135

--- Result #2 ---
  Article ID: 0557246012
  Product Name: Hood W Zip
  Color: Dark Red
  Description: Long-sleeved jacket in sweatshirt fabric with a lined drawstring hood, zip down the front, side pockets and ribbing at the cuffs and hem. Soft brushed inside.
  L2 Distance (Similarity): 0.6637

--- Result #3 ---
  Article ID: 0762930001
  Product Name: Glazing zip hoodie
  Color: Dark Pink
  Description: Jacket in soft, marled sweatshirt fabric with a double-layered drawstring hood, and a zip down the front and at the sides. Long raglan sleeves and ribbing at the cuffs and hem. Soft brushed inside.
  L2 Distance (Simil

# Model draft

In [2]:
# Install PyTorch Geometric with correct wheels
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 89.1 MB/s eta 0:00:0000:010:01
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 53.8 MB/s eta 0:00:0000:0100:01
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 42.9 MB/s eta 0:00:0000:0100:01
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:0000:01
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 50.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.7 MB/s eta 0:00:0000:01


In [3]:
import pandas as pd

# Load the data
articles_df = pd.read_csv('/kaggle/input/my-own/articles_cleaned.csv')
customers_df = pd.read_csv('/kaggle/input/my-own/customers_cleaned.csv')
transactions_df = pd.read_csv('/kaggle/input/my-own/transactions_cleaned.csv')

# For development on Kaggle, it's wise to start with a smaller sample
# Let's take transactions from the last N weeks to keep the graph manageable
transactions_df['t_dat'] = pd.to_datetime(transactions_df['t_dat'])
latest_date = transactions_df['t_dat'].max()
transactions_sample = transactions_df[transactions_df['t_dat'] > latest_date - pd.to_timedelta('14 days')]

# Filter customers and articles to only those present in the sample
active_articles = transactions_sample['article_id'].unique()
active_customers = transactions_sample['customer_id'].unique()

articles_df = articles_df[articles_df['article_id'].isin(active_articles)]
customers_df = customers_df[customers_df['customer_id'].isin(active_customers)]

print(f"Sampled Transactions: {len(transactions_sample)}")
print(f"Active Articles: {len(articles_df)}")
print(f"Active Customers: {len(customers_df)}")

Sampled Transactions: 480993
Active Articles: 22208
Active Customers: 127034


In [4]:
import torch
from torch_geometric.data import HeteroData

data = HeteroData()

# 1. Create Mappings for all node types to unique integer indices
# This is crucial because GNNs work with integer indices.

customer_mapping = {id: i for i, id in enumerate(customers_df['customer_id'].unique())}
article_mapping = {id: i for i, id in enumerate(articles_df['article_id'].unique())}
# ... and so on for all your attribute types
prod_group_mapping = {name: i for i, name in enumerate(articles_df['product_group_name'].unique())}
colour_group_mapping = {name: i for i, name in enumerate(articles_df['colour_group_name'].unique())}


# 2. Define Node Features (optional but recommended)
# For simplicity, we can start with learnable embeddings for each node.
# The size `embedding_dim` is a hyperparameter you choose.
embedding_dim = 64
data['customer'].x = torch.randn(len(customer_mapping), embedding_dim)
data['article'].x = torch.randn(len(article_mapping), embedding_dim)
data['product_group'].x = torch.randn(len(prod_group_mapping), embedding_dim)
data['colour_group'].x = torch.randn(len(colour_group_mapping), embedding_dim)

# 3. Create Edge Index Tensors for each relationship type

# Customer -> BOUGHT -> Article
cust_src = [customer_mapping[id] for id in transactions_sample['customer_id']]
art_dst = [article_mapping[id] for id in transactions_sample['article_id']]
data['customer', 'bought', 'article'].edge_index = torch.tensor([cust_src, art_dst])

# Article -> BELONGS_TO -> ProductGroup
art_src = [article_mapping[id] for id in articles_df['article_id']]
pg_dst = [prod_group_mapping[name] for name in articles_df['product_group_name']]
data['article', 'belongs_to', 'product_group'].edge_index = torch.tensor([art_src, pg_dst])

# Article -> HAS_COLOUR -> ColourGroup
art_src_color = [article_mapping[id] for id in articles_df['article_id']]
cg_dst = [colour_group_mapping[name] for name in articles_df['colour_group_name']]
data['article', 'has_colour', 'colour_group'].edge_index = torch.tensor([art_src_color, cg_dst])

# We also need the reverse edges for message passing in both directions
from torch_geometric.transforms import ToUndirected

# This creates reverse edges for all types, e.g., ('article', 'rev_bought', 'customer')
# It's a common practice to make the graph undirected for message passing.
data = ToUndirected()(data)

print(data)

HeteroData(
  customer={ x=[127034, 64] },
  article={ x=[22208, 64] },
  product_group={ x=[8, 64] },
  colour_group={ x=[48, 64] },
  (customer, bought, article)={ edge_index=[2, 480993] },
  (article, belongs_to, product_group)={ edge_index=[2, 22208] },
  (article, has_colour, colour_group)={ edge_index=[2, 22208] },
  (article, rev_bought, customer)={ edge_index=[2, 480993] },
  (product_group, rev_belongs_to, article)={ edge_index=[2, 22208] },
  (colour_group, rev_has_colour, article)={ edge_index=[2, 22208] }
)


In [5]:
import torch.nn as nn
from torch_geometric.nn import SAGEConv, HeteroConv, to_hetero

class HeteroGNN(nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = HeteroConv({
            # Define a SAGEConv for each edge type you want to learn from
            ('customer', 'bought', 'article'): SAGEConv((-1, -1), hidden_channels),
            ('article', 'belongs_to', 'product_group'): SAGEConv((-1, -1), hidden_channels),
            ('article', 'has_colour', 'colour_group'): SAGEConv((-1, -1), hidden_channels),
            
            # Reverse edges
            ('article', 'rev_bought', 'customer'): SAGEConv((-1, -1), hidden_channels),
            ('product_group', 'rev_belongs_to', 'article'): SAGEConv((-1, -1), hidden_channels),
            ('colour_group', 'rev_has_colour', 'article'): SAGEConv((-1, -1), hidden_channels),
        }, aggr='sum')

        self.conv2 = HeteroConv({
            ('customer', 'bought', 'article'): SAGEConv((-1, -1), hidden_channels),
            ('article', 'belongs_to', 'product_group'): SAGEConv((-1, -1), hidden_channels),
            ('article', 'has_colour', 'colour_group'): SAGEConv((-1, -1), hidden_channels),
            
            ('article', 'rev_bought', 'customer'): SAGEConv((-1, -1), hidden_channels),
            ('product_group', 'rev_belongs_to', 'article'): SAGEConv((-1, -1), hidden_channels),
            ('colour_group', 'rev_has_colour', 'article'): SAGEConv((-1, -1), hidden_channels),
        }, aggr='sum')

    def forward(self, x_dict, edge_index_dict):
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: x.relu() for key, x in x_dict.items()}
        x_dict = self.conv2(x_dict, edge_index_dict)
        return x_dict

# Instantiate the model
model = HeteroGNN(hidden_channels=64)
print(model)

HeteroGNN(
  (conv1): HeteroConv(num_relations=6)
  (conv2): HeteroConv(num_relations=6)
)


In [6]:
import torch
from torch_geometric.loader import LinkNeighborLoader
import torch.nn.functional as F

# Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Move model to device
model = model.to(device)

# 1. Create a loader for mini-batch training on the links we want to predict
edge_label_index = data['customer', 'bought', 'article'].edge_index
loader = LinkNeighborLoader(
    data,
    num_neighbors=[10, 5],  # Sample 10 neighbors for the first layer, 5 for the second
    edge_label_index=(('customer', 'article'), edge_label_index),
    batch_size=128,
    shuffle=True,
    neg_sampling_ratio=1.0 # For each positive edge, sample 1 negative one
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training Loop
def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)  # Move batch to device

        optimizer.zero_grad()
        
        # Get embeddings for the nodes in the current mini-batch
        out = model(batch.x_dict, batch.edge_index_dict)
        
        # Get embeddings for the source and destination nodes of the links
        src_emb = out['customer'][batch['customer', 'article'].edge_label_index[0]]
        dst_emb = out['article'][batch['customer', 'article'].edge_label_index[1]]
        
        # Predict the link probability
        pred = (src_emb * dst_emb).sum(dim=-1)
        
        loss = F.binary_cross_entropy_with_logits(
            pred, batch['customer', 'article'].edge_label.float()
        )
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        
    return total_loss / len(loader.dataset)

for epoch in range(1, 11):
    loss = train()
    print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}")


Using device: cuda
Epoch: 001, Loss: 0.8191
Epoch: 002, Loss: 0.3972
Epoch: 003, Loss: 0.3099
Epoch: 004, Loss: 0.2692
Epoch: 005, Loss: 0.2467
Epoch: 006, Loss: 0.2312
Epoch: 007, Loss: 0.2188
Epoch: 008, Loss: 0.2120
Epoch: 009, Loss: 0.2047
Epoch: 010, Loss: 0.1996


In [7]:
import torch
import numpy as np
import json

# 1. Define the device you are using (best practice)
# This will automatically use the GPU if available, otherwise fall back to CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 2. Make sure your model is on the correct device and in evaluation mode
model.to(device)
model.eval()

# 3. <<--- THIS IS THE FIX --- >>
# Move the entire graph data object to the same device as the model.
# PyG's .to() method conveniently moves all tensor attributes.
data = data.to(device)

# 4. Now, perform inference. All tensors are on the same device.
with torch.no_grad():
    final_embeddings = model(data.x_dict, data.edge_index_dict)

# The 'final_embeddings' dictionary now contains tensors that are on the GPU.

# 5. Save the embeddings and mappings
# IMPORTANT: NumPy cannot access GPU memory. You must move the tensors
# back to the CPU with .cpu() before converting them to NumPy arrays.

customer_embeddings = final_embeddings['customer']
article_embeddings = final_embeddings['article']

# Saving embeddings
np.save('customer_embeddings.npy', customer_embeddings.cpu().numpy())
np.save('article_embeddings.npy', article_embeddings.cpu().numpy())
print("Embeddings saved as .npy files.")


Using device: cuda
Embeddings saved as .npy files.


In [8]:
if all(isinstance(k, np.integer) for k in customer_mapping.keys()):
    customer_mapping__serializable = {int(k): v for k, v in customer_mapping.items()}
    print("Converted customer_mapping keys to standard int.")
else:
    customer_mapping_serializable = customer_mapping


# FIX FOR article_mapping
# The keys are numpy.int64, so we convert them to standard python int.
article_mapping_serializable = {int(k): v for k, v in article_mapping.items()}
print("Converted article_mapping keys to standard int.")


# Now save the converted dictionaries
with open('customer_mapping.json', 'w') as f:
    json.dump(customer_mapping_serializable, f)

with open('article_mapping.json', 'w') as f:
    json.dump(article_mapping_serializable, f)

print("Mappings saved as .json files.")

Converted article_mapping keys to standard int.
Mappings saved as .json files.


In [ ]:
# --- In a new Kaggle session, you would load them like this ---
# loaded_customer_embeddings = np.load('customer_embeddings.npy')
# loaded_article_embeddings = np.load('article_embeddings.npy')

# with open('customer_mapping.json', 'r') as f:
#     loaded_customer_mapping = json.load(f)

# with open('article_mapping.json', 'r') as f:
#     loaded_article_mapping = json.load(f)

In [9]:
# GNN Recommendation
def get_gnn_recommendations(customer_id, k=10):
    customer_idx = customer_mapping[customer_id]
    customer_emb = customer_embeddings[customer_idx]
    
    # Calculate cosine similarity between the user and all articles
    scores = torch.matmul(article_embeddings, customer_emb)
    
    # Get top k scores
    top_k_scores, top_k_indices = torch.topk(scores, k)
    
    # Convert article indices back to original IDs
    rev_article_mapping = {v: k for k, v in article_mapping.items()}
    recs = [rev_article_mapping[idx.item()] for idx in top_k_indices]
    
    return recs

print("GNN Recommendations for a sample customer:", get_gnn_recommendations(list(customer_mapping.keys())[0]))


# Baseline: Neighborhood Similarity ("Customers who bought this also bought...")
def get_baseline_recommendations(article_id, k=10):
    # Find customers who bought the target article
    customers_who_bought = transactions_df[transactions_df['article_id'] == article_id]['customer_id'].unique()
    
    # Find all other articles those customers bought
    other_articles = transactions_df[transactions_df['customer_id'].isin(customers_who_bought)]['article_id']
    
    # Exclude the original article and count frequencies
    recommendations = other_articles[other_articles != article_id].value_counts().nlargest(k).index.tolist()
    
    return recommendations

print("\nBaseline Recommendations for a sample article:", get_baseline_recommendations(list(article_mapping.keys())[0]))

GNN Recommendations for a sample customer: [794321007, 911870003, 874122024, 816832011, 903276002, 863595004, 685813042, 562613010, 851094007, 928461001]

Baseline Recommendations for a sample article: [767834002, 841383003, 783346001, 841383002, 565379002, 736870001, 749699002, 892910002, 610776001, 610776074]


**Block to run while demonstration**

In [13]:
import pandas as pd
import numpy as np
import json
import torch
import torch.nn.functional as F
from IPython.display import display, HTML

print("--- Loading all necessary assets for demonstration ---")

# --- 1. Load DataFrames ---
articles_df = pd.read_csv('/kaggle/input/my-own/articles_cleaned.csv')
transactions_df = pd.read_csv('/kaggle/input/my-own/transactions_cleaned.csv')

# --- 2. Load Embeddings and Mappings ---
# Load embeddings and convert them to PyTorch tensors for calculations
customer_embeddings = torch.from_numpy(np.load('/kaggle/working/customer_embeddings.npy'))
article_embeddings = torch.from_numpy(np.load('/kaggle/working/article_embeddings.npy'))

# Load mappings
with open('/kaggle/working/customer_mapping.json', 'r') as f:
    customer_mapping = json.load(f)

with open('/kaggle/working/article_mapping.json', 'r') as f:
    # JSON saves integer keys as strings, so we must convert them back
    article_mapping = {int(k): v for k, v in json.load(f).items()}

# Create reverse mappings to go from index -> original ID
idx_to_customer = {v: k for k, v in customer_mapping.items()}
idx_to_article = {v: k for k, v in article_mapping.items()}

print("Assets loaded successfully.\n")


# --- 3. Define Helper and Recommendation Functions ---

def display_customer_history(customer_id, num_items=5):
    """Looks up and displays the most recent purchases for a given customer with extended details."""
    display(HTML(f"<h3>Purchase History for Customer: <code>{customer_id}</code></h3>"))
    
    purchases = transactions_df[transactions_df['customer_id'] == customer_id]
    
    if purchases.empty:
        print("No purchase history found for this customer.")
        return
        
    purchase_details = articles_df.merge(
        purchases,
        on='article_id'
    ).sort_values(by='t_dat', ascending=False).head(num_items)
    
    # Define a comprehensive list of columns to display
    display_cols = [
        'prod_name',
        'product_type_name',
        'product_group_name',
        'colour_group_name',
        'department_name',
        'section_name',
        'garment_group_name',
        'detail_desc'
    ]
    
    # Ensure all columns exist before trying to display them
    existing_cols = [col for col in display_cols if col in purchase_details.columns]
    
    print("Recent purchases:")
    display(purchase_details[existing_cols])


def get_gnn_recommendations(customer_id, k=10):
    """
    Generates top-k GNN-based recommendations for a customer with extended details.
    Filters out items they have already purchased.
    """
    display(HTML(f"<h3>Top {k} GNN Recommendations for Customer: <code>{customer_id}</code></h3>"))

    if customer_id not in customer_mapping:
        print("Customer not found in the mapping.")
        return None

    # 1. Get the user's embedding
    customer_idx = customer_mapping[customer_id]
    user_embedding = customer_embeddings[customer_idx].unsqueeze(0)

    # 2. Calculate cosine similarity
    scores = F.cosine_similarity(user_embedding, article_embeddings)

    # 3. Filter out items the user has already bought
    purchased_article_ids = transactions_df[transactions_df['customer_id'] == customer_id]['article_id'].unique()
    purchased_indices = [article_mapping[aid] for aid in purchased_article_ids if aid in article_mapping]
    
    if purchased_indices:
        scores[purchased_indices] = -1.0

    # 4. Get top-k recommendations
    top_k_scores, top_k_indices = torch.topk(scores, k)
    
    # 5. Convert indices back to original article IDs
    recommended_article_ids = [idx_to_article[idx.item()] for idx in top_k_indices]
    
    # 6. Look up details and return a nice DataFrame
    rec_details = articles_df[articles_df['article_id'].isin(recommended_article_ids)].copy()
    rec_details['similarity_score'] = [round(s.item(), 4) for s in top_k_scores]
    
    rec_details = rec_details.set_index('article_id').loc[recommended_article_ids].reset_index()
    
    # Define the comprehensive list of columns to display for recommendations
    display_cols = [
        'prod_name',
        'product_type_name',
        'product_group_name',
        'colour_group_name',
        'perceived_colour_master_name',
        'department_name',
        'section_name',
        'garment_group_name',
        'detail_desc',
        'similarity_score'
    ]
    
    existing_cols = [col for col in display_cols if col in rec_details.columns]
    
    print("Recommended items:")
    display(rec_details[existing_cols])
    return rec_details


# --- 4. RUN THE DEMONSTRATION ---

# You can change the index to see recommendations for different users
# For example, try [10], [100], [500], etc.
customer_index_to_show = 150
sample_customer_id = list(customer_mapping.keys())[customer_index_to_show] 

# Show their purchase history for context
display_customer_history(sample_customer_id, num_items=5)

# Generate and show new recommendations for them
recommendations = get_gnn_recommendations(sample_customer_id, k=7)

--- Loading all necessary assets for demonstration ---
Assets loaded successfully.



Recent purchases:


,prod_name,product_type_name,product_group_name,colour_group_name,department_name,section_name,garment_group_name,detail_desc
0,Jen Bermuda denim shorts,Shorts,Garment Lower body,Light Blue,Shorts & Skirts,Womens Casual,Shorts,"5-pocket shorts in washed, stretch denim with ..."
1,HEAVEN shaping HW tight,Leggings/Tights,Garment Lower body,Dark Grey,Ladies Sport Bottoms,Ladies H&M Sport,Jersey Fancy,High-waisted sports tights in fast-drying func...
2,Norway hood jacket,Hoodie,Garment Upper body,Green,Ladies Sport Bras,Ladies H&M Sport,Jersey Fancy,Fitted jacket in fast-drying functional fabric...


Recommended items:


,prod_name,product_type_name,product_group_name,colour_group_name,perceived_colour_master_name,department_name,section_name,garment_group_name,detail_desc,similarity_score
0,Pink HW barrel,Trousers,Garment Lower body,Light Beige,Beige,Trouser,Womens Everyday Collection,Trousers,"5-pocket, ankle-length trousers in washed cott...",0.1325
1,Snow chino TVP RW TVP,Trousers,Garment Lower body,Black,Black,Trouser,Womens Casual,Trousers,Ankle-length chinos in a soft cotton weave wit...,0.1545
2,ELVIS JOGGER,Trousers,Garment Lower body,Green,Khaki green,Jersey,Womens Casual,Jersey Fancy,Joggers in sweatshirt fabric with an elasticat...,0.1271
3,CORAL LONG HOOD,Hoodie,Garment Upper body,Light Beige,White,Jersey,Womens Casual,Jersey Fancy,Oversized top in sweatshirt fabric with flatlo...,0.1368
4,Speedy conscious tee,T-shirt,Garment Upper body,Light Pink,Pink,Ladies Sport Bras,Ladies H&M Sport,Jersey Fancy,Straight-cut sports top in fast-drying mesh wi...,0.1470
5,Baraboom throw-on,Cardigan,Garment Upper body,Black,Black,Knitwear,Womens Everyday Collection,Knitwear,"Long cardigan in a soft, fine knit containing ...",0.1413
6,Perrie Slim Mom Denim TRS,Trousers,Garment Lower body,Light Blue,Blue,Trousers,Divided Collection,Trousers,"5-pocket, ankle-length jeans in washed, sturdy...",0.1846


In [ ]:
from neo4j import GraphDatabase

# --- First, you would need to load the graph structure into Neo4j ---
# This is typically done with Cypher queries, e.g.:
# UNWIND $customers as customer MERGE (c:Customer {id: customer.id})
# UNWIND $articles as article MERGE (a:Article {id: article.id, name: article.name})
# etc. for all nodes and relationships.

# --- Then, update nodes with their learned embeddings ---
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "your_password"
driver = GraphDatabase.driver(uri, auth=(user, password))

def store_embeddings(tx, node_label, embeddings, mapping):
    # Convert PyTorch tensor to a list of lists
    embeddings_list = embeddings.cpu().numpy().tolist()
    # Create a reverse mapping from index to original ID
    rev_mapping = {v: k for k, v in mapping.items()}
    
    for i, emb in enumerate(embeddings_list):
        original_id = rev_mapping[i]
        query = (
            f"MATCH (n:{node_label} {{id: $id}}) "
            "SET n.embedding = $embedding"
        )
        tx.run(query, id=original_id, embedding=emb)

with driver.session() as session:
    session.write_transaction(store_embeddings, "Customer", customer_embeddings, customer_mapping)
    session.write_transaction(store_embeddings, "Article", article_embeddings, article_mapping)
    
driver.close()